In [ ]:
import os
import sys
import argparse
import datetime
import time
import csv
import os.path as osp
import numpy as np
import warnings
import errno
import importlib
import pandas as pd

In [ ]:

warnings.filterwarnings('ignore')

In [ ]:
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision

In [ ]:
"""Base augmentations operators."""

import numpy as np
from PIL import Image, ImageOps, ImageEnhance

# ImageNet code should change this value
IMAGE_SIZE = 224


def int_parameter(level, maxval):
  """Helper function to scale `val` between 0 and maxval .
  Args:
    level: Level of the operation that will be between [0, `PARAMETER_MAX`].
    maxval: Maximum value that the operation can have. This will be scaled to
      level/PARAMETER_MAX.
  Returns:
    An int that results from scaling `maxval` according to `level`.
  """
  return int(level * maxval / 10)


def float_parameter(level, maxval):
  """Helper function to scale `val` between 0 and maxval.
  Args:
    level: Level of the operation that will be between [0, `PARAMETER_MAX`].
    maxval: Maximum value that the operation can have. This will be scaled to
      level/PARAMETER_MAX.
  Returns:
    A float that results from scaling `maxval` according to `level`.
  """
  return float(level) * maxval / 10.


def sample_level(n):
  return np.random.uniform(low=0.1, high=n)


def autocontrast(pil_img, _):
  return ImageOps.autocontrast(pil_img)


def equalize(pil_img, _):
  return ImageOps.equalize(pil_img)


def posterize(pil_img, level):
  level = int_parameter(sample_level(level), 4)
  return ImageOps.posterize(pil_img, 4 - level)


def rotate(pil_img, level):
  degrees = int_parameter(sample_level(level), 30)
  if np.random.uniform() > 0.5:
    degrees = -degrees
  return pil_img.rotate(degrees, resample=Image.BILINEAR)


def solarize(pil_img, level):
  level = int_parameter(sample_level(level), 256)
  return ImageOps.solarize(pil_img, 256 - level)


def shear_x(pil_img, level):
  level = float_parameter(sample_level(level), 0.3)
  if np.random.uniform() > 0.5:
    level = -level
  return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                           Image.AFFINE, (1, level, 0, 0, 1, 0),
                           resample=Image.BILINEAR)


def shear_y(pil_img, level):
  level = float_parameter(sample_level(level), 0.3)
  if np.random.uniform() > 0.5:
    level = -level
  return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                           Image.AFFINE, (1, 0, 0, level, 1, 0),
                           resample=Image.BILINEAR)


def translate_x(pil_img, level):
  level = int_parameter(sample_level(level), IMAGE_SIZE / 3)
  if np.random.random() > 0.5:
    level = -level
  return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                           Image.AFFINE, (1, 0, level, 0, 1, 0),
                           resample=Image.BILINEAR)


def translate_y(pil_img, level):
  level = int_parameter(sample_level(level), IMAGE_SIZE / 3)
  if np.random.random() > 0.5:
    level = -level
  return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                           Image.AFFINE, (1, 0, 0, 0, 1, level),
                           resample=Image.BILINEAR)


# operation that overlaps with ImageNet-C's test set
def color(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Color(pil_img).enhance(level)


# operation that overlaps with ImageNet-C's test set
def contrast(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Contrast(pil_img).enhance(level)


# operation that overlaps with ImageNet-C's test set
def brightness(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Brightness(pil_img).enhance(level)


# operation that overlaps with ImageNet-C's test set
def sharpness(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Sharpness(pil_img).enhance(level)


augmentations = [
    autocontrast, equalize, posterize, rotate, solarize, shear_x, shear_y,
    translate_x, translate_y
]

augmentations_all = [
    autocontrast, equalize, posterize, rotate, solarize, shear_x, shear_y,
    translate_x, translate_y, color, contrast, brightness, sharpness
]

In [ ]:
import io
import random
from PIL import Image
import numpy as np

class APRecombination(object):
    def __init__(self, img_size=224, aug=None):
        if aug is None:
            augmentations.IMAGE_SIZE = img_size
            self.aug_list = augmentations.augmentations
        else:
            self.aug_list = aug.augmentations

    def __call__(self, x):
        '''
        :param img: (PIL Image): Image
        :return: code img (PIL Image): Image
        '''

        op = np.random.choice(self.aug_list)
        x = op(x, 3)

        p = random.uniform(0, 1)
        if p > 0.5:
            return x

        x_aug = x.copy()
        op = np.random.choice(self.aug_list)
        x_aug = op(x_aug, 3)

        x = np.array(x).astype(np.uint8) 
        x_aug = np.array(x_aug).astype(np.uint8)
        
        fft_1 = np.fft.fftshift(np.fft.fftn(x))
        fft_2 = np.fft.fftshift(np.fft.fftn(x_aug))
        
        abs_1, angle_1 = np.abs(fft_1), np.angle(fft_1)
        abs_2, angle_2 = np.abs(fft_2), np.angle(fft_2)

        fft_1 = abs_1*np.exp((1j) * angle_2)
        fft_2 = abs_2*np.exp((1j) * angle_1)

        p = random.uniform(0, 1)

        if p > 0.5:
            x = np.fft.ifftn(np.fft.ifftshift(fft_1))
        else:
            x = np.fft.ifftn(np.fft.ifftshift(fft_2))

        x = x.astype(np.uint8)
        x = Image.fromarray(x)
        
        return x

In [ ]:
import torch
from torchvision import transforms

normalize = transforms.Compose([
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]),
    ])

def train_transforms(_transforms):
    transforms_list = []
    if 'aprs' in _transforms:
        print('APRecombination', _transforms)
        transforms_list.extend([
            transforms.RandomApply([APRecombination()], p=1.0),
            transforms.RandomCrop(32, padding=4, fill=128),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ])
    else:
        transforms_list.extend([
            transforms.RandomCrop(32, padding=4, fill=128),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ])

    return transforms_list


def test_transforms():
    test_transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    return test_transform

In [ ]:
!pip install kaggle
from google.colab import files
files.upload() #put your kaggle api token here

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"harshgujarathi","key":"5015cf381c11cbc848aaa40f4a916ee2"}'}

In [ ]:
#@title
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d gpiosenka/balls-image-classification
!unzip /content/balls-image-classification.zip

 92% 133M/144M [00:01<00:00, 113MB/s]
100% 144M/144M [00:01<00:00, 131MB/s]
Archive:  /content/balls-image-classification.zip
  inflating: EfficientNetB0 model-30-(224 X 224)- 99.33.h5  
  inflating: balls.csv               
  inflating: test/baseball/1.jpg     
  inflating: test/baseball/2.jpg     
  inflating: test/baseball/3.jpg     
  inflating: test/baseball/4.jpg     
  inflating: test/baseball/5.jpg     
  inflating: test/basketball/1.jpg   
  inflating: test/basketball/2.jpg   
  inflating: test/basketball/3.jpg   
  inflating: test/basketball/4.jpg   
  inflating: test/basketball/5.jpg   
  inflating: test/beachballs/1.jpg   
  inflating: test/beachballs/2.jpg   
  inflating: test/beachballs/3.jpg   
  inflating: test/beachballs/4.jpg   
  inflating: test/beachballs/5.jpg   
  inflating: test/billiard ball/1.jpg  
  inflating: test/billiard ball/2.jpg  
  inflating: test/billiard ball/3.jpg  
  inflating: test/billiard ball/4.jpg  
  inflating: test/billiard ball/5.jpg  
  inf

In [ ]:
import os
import cv2
import pickle
import numpy as np
from scipy import signal
from PIL import Image

import torch
from torchvision import transforms
from torchvision.datasets import CIFAR10, CIFAR100, ImageFolder,ImageNet


class CIFARC(CIFAR10):
    def __init__(
            self,
            root,
            key = 'zoom_blur',
            transform = None,
            target_transform = None,
    ):

        super(CIFAR10, self).__init__(root, transform=transform,
                                      target_transform=target_transform)

        data_path = os.path.join(root, key+'.npy')
        labels_path = os.path.join(root, 'labels.npy')

        self.data = np.load(data_path)
        self.targets = np.load(labels_path)

    def __getitem__(self, index: int):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.targets[index]

        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

class Balls(object):
    def __init__(self, dataroot='', use_gpu=True, num_workers=4, batch_size=16, _transforms='', _eval=False):

        transforms_list = train_transforms(_transforms)

        train_transform = transforms.Compose(transforms_list)
        test_transform = test_transforms()
        self.train_transform = train_transform

        pin_memory = True if use_gpu else False

        trainset = ImageFolder(root=(os.path.join(dataroot,'train')),transform=train_transform)
        
        self.train_loader = torch.utils.data.DataLoader(
            trainset, batch_size=batch_size, shuffle=True,
            num_workers=num_workers, pin_memory=pin_memory,
        )
        
        testset =  ImageFolder(root=(os.path.join(dataroot,'test')),transform=test_transform)
        
        self.test_loader = torch.utils.data.DataLoader(
            testset, batch_size=batch_size, shuffle=False,
            num_workers=num_workers, pin_memory=pin_memory,
        )

        if _eval:
            self.out_loaders = dict()
            self.out_keys = ['gaussian_noise', 'shot_noise', 'impulse_noise', 'defocus_blur',
                            'glass_blur', 'motion_blur', 'zoom_blur', 'snow', 'frost', 'fog',
                            'brightness', 'contrast', 'elastic_transform', 'pixelate',
                            'jpeg_compression']

            data_root = os.path.join(dataroot, 'test')
            for key in self.out_keys:
                outset = ImageFolder(root=(os.path.join(dataroot,'test')),transform=test_transform)
                out_loader = torch.utils.data.DataLoader(
                    outset, batch_size=batch_size, shuffle=False,
                    num_workers=num_workers, pin_memory=pin_memory,
                )
                self.out_loaders[key] = out_loader
        
        self.num_classes = 30

class Balls2(object):
    def __init__(self, dataroot='', use_gpu=True, num_workers=4, batch_size=128, _transforms='', _eval=False):

        transforms_list = train_transforms(_transforms)

        train_transform = transforms.Compose(transforms_list)
        test_transform = test_transforms()
        self.train_transform = train_transform

        pin_memory = True if use_gpu else False


        trainset =ImageFolder(root=(os.path.join(dataroot,'valid')),transform=train_transform)
        
        self.train_loader = torch.utils.data.DataLoader(
            trainset, batch_size=batch_size, shuffle=True,
            num_workers=num_workers, pin_memory=pin_memory,
        )
        
        testset =  ImageFolder(root=(os.path.join(dataroot,'test')),transform=test_transform)
        
        self.test_loader = torch.utils.data.DataLoader(
            testset, batch_size=batch_size, shuffle=False,
            num_workers=num_workers, pin_memory=pin_memory,
        )

        if _eval:
            self.out_loaders = dict()
            self.out_keys = ['gaussian_noise', 'shot_noise', 'impulse_noise', 'defocus_blur',
                            'glass_blur', 'motion_blur', 'zoom_blur', 'snow', 'frost', 'fog',
                            'brightness', 'contrast', 'elastic_transform', 'pixelate',
                            'jpeg_compression']

            data_root = os.path.join(dataroot, 'test')
            for key in self.out_keys:
                outset = ImageFolder(root=(os.path.join(dataroot,'test')),transform=test_transform)
                out_loader = torch.utils.data.DataLoader(
                    outset, batch_size=batch_size, shuffle=False,
                    num_workers=num_workers, pin_memory=pin_memory,
                )
                self.out_loaders[key] = out_loader

        self.num_classes = 30

In [ ]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""AllConv implementation (https://arxiv.org/abs/1412.6806)."""
import math
import torch
import torch.nn as nn


class GELU(nn.Module):

  def forward(self, x):
    return torch.sigmoid(1.702 * x) * x


def make_layers(cfg):
  """Create a single layer."""
  layers = []
  in_channels = 3
  for v in cfg:
    if v == 'Md':
      layers += [nn.MaxPool2d(kernel_size=2, stride=2), nn.Dropout(p=0.5)]
    elif v == 'A':
      layers += [nn.AvgPool2d(kernel_size=8)]
    elif v == 'NIN':
      conv2d = nn.Conv2d(in_channels, in_channels, kernel_size=1, padding=1)
      layers += [conv2d, nn.BatchNorm2d(in_channels), GELU()]
    elif v == 'nopad':
      conv2d = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=0)
      layers += [conv2d, nn.BatchNorm2d(in_channels), GELU()]
    else:
      conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
      layers += [conv2d, nn.BatchNorm2d(v), GELU()]
      in_channels = v
  return nn.Sequential(*layers)


class AllConvNet(nn.Module):
  """AllConvNet main class."""

  def __init__(self, num_classes):
    super(AllConvNet, self).__init__()

    self.num_classes = num_classes
    self.width1, w1 = 96, 96
    self.width2, w2 = 192, 192

    self.features = make_layers(
        [w1, w1, w1, 'Md', w2, w2, w2, 'Md', 'nopad', 'NIN', 'NIN', 'A'])
    self.classifier = nn.Linear(self.width2, num_classes)

    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))  # He initialization
      elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
      elif isinstance(m, nn.Linear):
        m.bias.data.zero_()

  def forward(self, x, rf=False, _eval=False):
    if _eval:
        # switch to eval mode
        self.eval()
    else:
        self.train()

    x = self.features(x)
    x = x.view(x.size(0), -1)
    y = self.classifier(x)
    if rf:
        return x, y
    return y

In [ ]:
"""DenseNet implementation (https://arxiv.org/abs/1608.06993)."""
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


class Bottleneck(nn.Module):
  """Bottleneck block for DenseNet."""

  def __init__(self, n_channels, growth_rate):
    super(Bottleneck, self).__init__()
    inter_channels = 4 * growth_rate
    self.bn1 = nn.BatchNorm2d(n_channels)
    self.conv1 = nn.Conv2d(
        n_channels, inter_channels, kernel_size=1, bias=False)
    self.bn2 = nn.BatchNorm2d(inter_channels)
    self.conv2 = nn.Conv2d(
        inter_channels, growth_rate, kernel_size=3, padding=1, bias=False)

  def forward(self, x):
    out = self.conv1(F.relu(self.bn1(x)))
    out = self.conv2(F.relu(self.bn2(out)))
    out = torch.cat((x, out), 1)
    return out


class SingleLayer(nn.Module):
  """Layer container for blocks."""

  def __init__(self, n_channels, growth_rate):
    super(SingleLayer, self).__init__()
    self.bn1 = nn.BatchNorm2d(n_channels)
    self.conv1 = nn.Conv2d(
        n_channels, growth_rate, kernel_size=3, padding=1, bias=False)

  def forward(self, x):
    out = self.conv1(F.relu(self.bn1(x)))
    out = torch.cat((x, out), 1)
    return out


class Transition(nn.Module):
  """Transition block."""

  def __init__(self, n_channels, n_out_channels):
    super(Transition, self).__init__()
    self.bn1 = nn.BatchNorm2d(n_channels)
    self.conv1 = nn.Conv2d(
        n_channels, n_out_channels, kernel_size=1, bias=False)

  def forward(self, x):
    out = self.conv1(F.relu(self.bn1(x)))
    out = F.avg_pool2d(out, 2)
    return out


class DenseNet(nn.Module):
  """DenseNet main class."""

  def __init__(self, growth_rate, depth, reduction, n_classes, bottleneck):
    super(DenseNet, self).__init__()

    if bottleneck:
      n_dense_blocks = int((depth - 4) / 6)
    else:
      n_dense_blocks = int((depth - 4) / 3)

    n_channels = 2 * growth_rate
    self.conv1 = nn.Conv2d(3, n_channels, kernel_size=3, padding=1, bias=False)

    self.dense1 = self._make_dense(n_channels, growth_rate, n_dense_blocks,
                                   bottleneck)
    n_channels += n_dense_blocks * growth_rate
    n_out_channels = int(math.floor(n_channels * reduction))
    self.trans1 = Transition(n_channels, n_out_channels)

    n_channels = n_out_channels
    self.dense2 = self._make_dense(n_channels, growth_rate, n_dense_blocks,
                                   bottleneck)
    n_channels += n_dense_blocks * growth_rate
    n_out_channels = int(math.floor(n_channels * reduction))
    self.trans2 = Transition(n_channels, n_out_channels)

    n_channels = n_out_channels
    self.dense3 = self._make_dense(n_channels, growth_rate, n_dense_blocks,
                                   bottleneck)
    n_channels += n_dense_blocks * growth_rate

    self.bn1 = nn.BatchNorm2d(n_channels)
    self.fc = nn.Linear(n_channels, n_classes)

    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
      elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
      elif isinstance(m, nn.Linear):
        m.bias.data.zero_()

  def _make_dense(self, n_channels, growth_rate, n_dense_blocks, bottleneck):
    layers = []
    for _ in range(int(n_dense_blocks)):
      if bottleneck:
        layers.append(Bottleneck(n_channels, growth_rate))
      else:
        layers.append(SingleLayer(n_channels, growth_rate))
      n_channels += growth_rate
    return nn.Sequential(*layers)

  def forward(self, x, rf=False, _eval=False):
    if _eval:
        # switch to eval mode
        self.eval()
    else:
        self.train()
    out = self.conv1(x)
    out = self.trans1(self.dense1(out))
    out = self.trans2(self.dense2(out))
    out = self.dense3(out)
    out = torch.squeeze(F.avg_pool2d(F.relu(self.bn1(out)), 8))
    y = self.fc(out)
    if rf:
        return out, y
    return y

def densenet(growth_rate=12, depth=40, num_classes=10):
  model = DenseNet(growth_rate, depth, 1., num_classes, False)
  return model

In [ ]:
"""ResNeXt implementation (https://arxiv.org/abs/1611.05431)."""
import math
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F


class ResNeXtBottleneck(nn.Module):
  """ResNeXt Bottleneck Block type C (https://github.com/facebookresearch/ResNeXt/blob/master/models/resnext.lua)."""
  expansion = 4

  def __init__(self,
               inplanes,
               planes,
               cardinality,
               base_width,
               stride=1,
               downsample=None):
    super(ResNeXtBottleneck, self).__init__()

    dim = int(math.floor(planes * (base_width / 64.0)))

    self.conv_reduce = nn.Conv2d(
        inplanes,
        dim * cardinality,
        kernel_size=1,
        stride=1,
        padding=0,
        bias=False)
    self.bn_reduce = nn.BatchNorm2d(dim * cardinality)

    self.conv_conv = nn.Conv2d(
        dim * cardinality,
        dim * cardinality,
        kernel_size=3,
        stride=stride,
        padding=1,
        groups=cardinality,
        bias=False)
    self.bn = nn.BatchNorm2d(dim * cardinality)

    self.conv_expand = nn.Conv2d(
        dim * cardinality,
        planes * 4,
        kernel_size=1,
        stride=1,
        padding=0,
        bias=False)
    self.bn_expand = nn.BatchNorm2d(planes * 4)

    self.downsample = downsample

  def forward(self, x):
    residual = x

    bottleneck = self.conv_reduce(x)
    bottleneck = F.relu(self.bn_reduce(bottleneck), inplace=True)

    bottleneck = self.conv_conv(bottleneck)
    bottleneck = F.relu(self.bn(bottleneck), inplace=True)

    bottleneck = self.conv_expand(bottleneck)
    bottleneck = self.bn_expand(bottleneck)

    if self.downsample is not None:
      residual = self.downsample(x)

    return F.relu(residual + bottleneck, inplace=True)


class CifarResNeXt(nn.Module):
  """ResNext optimized for the Cifar dataset, as specified in https://arxiv.org/pdf/1611.05431.pdf."""

  def __init__(self, block, depth, cardinality, base_width, num_classes):
    super(CifarResNeXt, self).__init__()

    # Model type specifies number of layers for CIFAR-10 and CIFAR-100 model
    assert (depth - 2) % 9 == 0, 'depth should be one of 29, 38, 47, 56, 101'
    layer_blocks = (depth - 2) // 9

    self.cardinality = cardinality
    self.base_width = base_width
    self.num_classes = num_classes

    self.conv_1_3x3 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
    self.bn_1 = nn.BatchNorm2d(64)

    self.inplanes = 64
    self.stage_1 = self._make_layer(block, 64, layer_blocks, 1)
    self.stage_2 = self._make_layer(block, 128, layer_blocks, 2)
    self.stage_3 = self._make_layer(block, 256, layer_blocks, 2)
    self.avgpool = nn.AvgPool2d(8)
    self.classifier = nn.Linear(256 * block.expansion, num_classes)

    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
      elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
      elif isinstance(m, nn.Linear):
        init.kaiming_normal(m.weight)
        m.bias.data.zero_()

  def _make_layer(self, block, planes, blocks, stride=1):
    downsample = None
    if stride != 1 or self.inplanes != planes * block.expansion:
      downsample = nn.Sequential(
          nn.Conv2d(
              self.inplanes,
              planes * block.expansion,
              kernel_size=1,
              stride=stride,
              bias=False),
          nn.BatchNorm2d(planes * block.expansion),
      )

    layers = []
    layers.append(
        block(self.inplanes, planes, self.cardinality, self.base_width, stride,
              downsample))
    self.inplanes = planes * block.expansion
    for _ in range(1, blocks):
      layers.append(
          block(self.inplanes, planes, self.cardinality, self.base_width))

    return nn.Sequential(*layers)

  def forward(self, x, rf=False, _eval=False):
    if _eval:
        # switch to eval mode
        self.eval()
    else:
        self.train()
    x = self.conv_1_3x3(x)
    x = F.relu(self.bn_1(x), inplace=True)
    x = self.stage_1(x)
    x = self.stage_2(x)
    x = self.stage_3(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    y = self.classifier(x)

    if rf:
        return x, y
    return y

def resnext29(num_classes=10, cardinality=4, base_width=32):
  model = CifarResNeXt(ResNeXtBottleneck, 29, cardinality, base_width,
                       num_classes)
  return model

In [ ]:
"""WideResNet implementation (https://arxiv.org/abs/1605.07146)."""
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
  """Basic ResNet block."""

  def __init__(self, in_planes, out_planes, stride, drop_rate=0.0):
    super(BasicBlock, self).__init__()
    self.bn1 = nn.BatchNorm2d(in_planes)
    self.relu1 = nn.ReLU(inplace=True)
    self.conv1 = nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False)
    self.bn2 = nn.BatchNorm2d(out_planes)
    self.relu2 = nn.ReLU(inplace=True)
    self.conv2 = nn.Conv2d(
        out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.drop_rate = drop_rate
    self.is_in_equal_out = (in_planes == out_planes)
    self.conv_shortcut = (not self.is_in_equal_out) and nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=1,
        stride=stride,
        padding=0,
        bias=False) or None

  def forward(self, x):
    if not self.is_in_equal_out:
      x = self.relu1(self.bn1(x))
    else:
      out = self.relu1(self.bn1(x))
    if self.is_in_equal_out:
      out = self.relu2(self.bn2(self.conv1(out)))
    else:
      out = self.relu2(self.bn2(self.conv1(x)))
    if self.drop_rate > 0:
      out = F.dropout(out, p=self.drop_rate, training=self.training)
    out = self.conv2(out)
    if not self.is_in_equal_out:
      return torch.add(self.conv_shortcut(x), out)
    else:
      return torch.add(x, out)


class NetworkBlock(nn.Module):
  """Layer container for blocks."""

  def __init__(self,
               nb_layers,
               in_planes,
               out_planes,
               block,
               stride,
               drop_rate=0.0):
    super(NetworkBlock, self).__init__()
    self.layer = self._make_layer(block, in_planes, out_planes, nb_layers,
                                  stride, drop_rate)

  def _make_layer(self, block, in_planes, out_planes, nb_layers, stride,
                  drop_rate):
    layers = []
    for i in range(nb_layers):
      layers.append(
          block(i == 0 and in_planes or out_planes, out_planes,
                i == 0 and stride or 1, drop_rate))
    return nn.Sequential(*layers)

  def forward(self, x):
    return self.layer(x)


class WideResNet(nn.Module):
  """WideResNet class."""

  def __init__(self, depth, num_classes, widen_factor=1, drop_rate=0.0):
    super(WideResNet, self).__init__()
    n_channels = [16, 16 * widen_factor, 32 * widen_factor, 64 * widen_factor]
    assert (depth - 4) % 6 == 0
    n = (depth - 4) // 6
    block = BasicBlock
    # 1st conv before any network block
    self.conv1 = nn.Conv2d(
        3, n_channels[0], kernel_size=3, stride=1, padding=1, bias=False)
    # 1st block
    self.block1 = NetworkBlock(n, n_channels[0], n_channels[1], block, 1,
                               drop_rate)
    # 2nd block
    self.block2 = NetworkBlock(n, n_channels[1], n_channels[2], block, 2,
                               drop_rate)
    # 3rd block
    self.block3 = NetworkBlock(n, n_channels[2], n_channels[3], block, 2,
                               drop_rate)
    # global average pooling and classifier
    self.bn1 = nn.BatchNorm2d(n_channels[3])
    self.relu = nn.ReLU(inplace=True)
    self.fc = nn.Linear(n_channels[3], num_classes)
    self.n_channels = n_channels[3]

    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
      elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
      elif isinstance(m, nn.Linear):
        m.bias.data.zero_()

  def forward(self, x, rf=False, _eval=False):
    if _eval:
        # switch to eval mode
        self.eval()
    else:
        self.train()

    out = self.conv1(x)
    out = self.block1(out)
    out = self.block2(out)
    out = self.block3(out)
    out = self.relu(self.bn1(out))
    out = F.avg_pool2d(out, 8)
    out = out.view(-1, self.n_channels)
    y =  self.fc(out)

    if rf:
        return out, y
    return y

In [ ]:
import os
import cv2
import os.path as osp
import numpy as np
from PIL import Image
import torch.fft
import torch
from torch.autograd import Variable
import torch.nn.functional as F


def test(net, criterion, testloader, outloader, attack=None, epoch=None, **options):
    net.eval()
    correct, total, adv_correct = 0, 0, 0

    torch.cuda.empty_cache()

    _pred_k, _pred_u, _labels = [], [], []

    with torch.no_grad():
        for data, labels in testloader:
            if options['use_gpu']:
                data, labels = data.cuda(), labels.cuda()             
                data = normalize(data)
                logits = net(data, _eval=True)
                predictions = logits.data.max(1)[1]
                total += labels.size(0)
                correct += (predictions == labels.data).sum()
            
                _pred_k.append(logits.data.cpu().numpy())
                _labels.append(labels.data.cpu().numpy())

        for batch_idx, (data, labels) in enumerate(outloader):
            if options['use_gpu']:
                data, labels = data.cuda(), labels.cuda()
                data = normalize(data)
            with torch.set_grad_enabled(False):
                logits = net(data, _eval=True)
                _pred_u.append(logits.data.cpu().numpy())

    _pred_k = np.concatenate(_pred_k, 0)
    _pred_u = np.concatenate(_pred_u, 0)
    _labels = np.concatenate(_labels, 0)
    
    # # Out-of-Distribution detction evaluation
    x1, x2 = np.max(_pred_k, axis=1), np.max(_pred_u, axis=1)
    results = metric_ood(x1, x2)['Bas']

    # Accuracy
    acc = float(correct) * 100. / float(total)
    results['ACC'] = acc

    print('Acc: {:.5f}'.format(acc))

    return results

def test_robustness(net, criterion, testloader, epoch=None, label='', **options):
    net.eval()
    results = dict()
    correct, total = 0, 0

    torch.cuda.empty_cache()

    with torch.no_grad():
        for data, labels in testloader:
            if options['use_gpu']:
                data, labels = data.cuda(), labels.cuda()
                data = normalize(data)
            with torch.set_grad_enabled(False):
                logits = net(data, _eval=True)
                predictions = logits.data.max(1)[1]
                total += labels.size(0)
                correct += (predictions == labels.data).sum()

    # Accuracy
    acc = float(correct) * 100. / float(total)
    results['ACC'] = acc

    return results

In [ ]:
import os
import sys
import numpy as np

def get_curve_online(known, novel, stypes = ['Bas']):
    tp, fp = dict(), dict()
    tnr_at_tpr95 = dict()
    for stype in stypes:
        known.sort()
        novel.sort()
        end = np.max([np.max(known), np.max(novel)])
        start = np.min([np.min(known),np.min(novel)])
        num_k = known.shape[0]
        num_n = novel.shape[0]
        tp[stype] = -np.ones([num_k+num_n+1], dtype=int)
        fp[stype] = -np.ones([num_k+num_n+1], dtype=int)
        tp[stype][0], fp[stype][0] = num_k, num_n
        k, n = 0, 0
        for l in range(num_k+num_n):
            if k == num_k:
                tp[stype][l+1:] = tp[stype][l]
                fp[stype][l+1:] = np.arange(fp[stype][l]-1, -1, -1)
                break
            elif n == num_n:
                tp[stype][l+1:] = np.arange(tp[stype][l]-1, -1, -1)
                fp[stype][l+1:] = fp[stype][l]
                break
            else:
                if novel[n] < known[k]:
                    n += 1
                    tp[stype][l+1] = tp[stype][l]
                    fp[stype][l+1] = fp[stype][l] - 1
                else:
                    k += 1
                    tp[stype][l+1] = tp[stype][l] - 1
                    fp[stype][l+1] = fp[stype][l]
        tpr95_pos = np.abs(tp[stype] / num_k - .95).argmin()
        tnr_at_tpr95[stype] = 1. - fp[stype][tpr95_pos] / num_n
    return tp, fp, tnr_at_tpr95

def metric_ood(x1, x2, stypes = ['Bas'], verbose=True):
    tp, fp, tnr_at_tpr95 = get_curve_online(x1, x2, stypes)
    results = dict()
    mtypes = ['TNR', 'AUROC', 'DTACC', 'AUIN', 'AUOUT']
    if verbose:
        print('      ', end='')
        for mtype in mtypes:
            print(' {mtype:6s}'.format(mtype=mtype), end='')
        print('')
        
    for stype in stypes:
        if verbose:
            print('{stype:5s} '.format(stype=stype), end='')
        results[stype] = dict()
        
        # TNR
        mtype = 'TNR'
        results[stype][mtype] = 100.*tnr_at_tpr95[stype]
        if verbose:
            print(' {val:6.3f}'.format(val=results[stype][mtype]), end='')
        
        # AUROC
        mtype = 'AUROC'
        tpr = np.concatenate([[1.], tp[stype]/tp[stype][0], [0.]])
        fpr = np.concatenate([[1.], fp[stype]/fp[stype][0], [0.]])
        results[stype][mtype] = 100.*(-np.trapz(1.-fpr, tpr))
        if verbose:
            print(' {val:6.3f}'.format(val=results[stype][mtype]), end='')
        
        # DTACC
        mtype = 'DTACC'
        results[stype][mtype] = 100.*(.5 * (tp[stype]/tp[stype][0] + 1.-fp[stype]/fp[stype][0]).max())
        if verbose:
            print(' {val:6.3f}'.format(val=results[stype][mtype]), end='')
        
        # AUIN
        mtype = 'AUIN'
        denom = tp[stype]+fp[stype]
        denom[denom == 0.] = -1.
        pin_ind = np.concatenate([[True], denom > 0., [True]])
        pin = np.concatenate([[.5], tp[stype]/denom, [0.]])
        results[stype][mtype] = 100.*(-np.trapz(pin[pin_ind], tpr[pin_ind]))
        if verbose:
            print(' {val:6.3f}'.format(val=results[stype][mtype]), end='')
        
        # AUOUT
        mtype = 'AUOUT'
        denom = tp[stype][0]-tp[stype]+fp[stype][0]-fp[stype]
        denom[denom == 0.] = -1.
        pout_ind = np.concatenate([[True], denom > 0., [True]])
        pout = np.concatenate([[0.], (fp[stype][0]-fp[stype])/denom, [.5]])
        results[stype][mtype] = 100.*(np.trapz(pout[pout_ind], 1.-fpr[pout_ind]))
        if verbose:
            print(' {val:6.3f}'.format(val=results[stype][mtype]), end='')
            print('')
    
    return results

def compute_oscr(pred_k, pred_u, labels):
    x1, x2 = np.max(pred_k, axis=1), np.max(pred_u, axis=1)
    pred = np.argmax(pred_k, axis=1)
    correct = (pred == labels)
    m_x1 = np.zeros(len(x1))
    m_x1[pred == labels] = 1
    k_target = np.concatenate((m_x1, np.zeros(len(x2))), axis=0)
    u_target = np.concatenate((np.zeros(len(x1)), np.ones(len(x2))), axis=0)
    predict = np.concatenate((x1, x2), axis=0)
    n = len(predict)

    # Cutoffs are of prediction values
    
    CCR = [0 for x in range(n+2)]
    FPR = [0 for x in range(n+2)] 

    idx = predict.argsort()

    s_k_target = k_target[idx]
    s_u_target = u_target[idx]

    for k in range(n-1):
        CC = s_k_target[k+1:].sum()
        FP = s_u_target[k:].sum()

        # True	Positive Rate
        CCR[k] = float(CC) / float(len(x1))
        # False Positive Rate
        FPR[k] = float(FP) / float(len(x2))

    CCR[n] = 0.0
    FPR[n] = 0.0
    CCR[n+1] = 1.0
    FPR[n+1] = 1.0

    # Positions of ROC curve (FPR, TPR)
    ROC = sorted(zip(FPR, CCR), reverse=True)

    OSCR = 0

    # Compute AUROC Using Trapezoidal Rule
    for j in range(n+1):
        h =   ROC[j][0] - ROC[j+1][0]
        w =  (ROC[j][1] + ROC[j+1][1]) / 2.0

        OSCR = OSCR + h*w

    return OSCR


In [ ]:
def mkdir_if_missing(directory):
    if not osp.exists(directory):
        try:
            os.makedirs(directory)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
def save_networks(networks, result_dir, name='', loss='', criterion=None):
    mkdir_if_missing(osp.join(result_dir, 'checkpoints'))
    weights = networks.state_dict()
    filename = '{}/checkpoints/{}_{}.pth'.format(result_dir, name, loss)
    torch.save(weights, filename)
    if criterion:
        weights = criterion.state_dict()
        filename = '{}/checkpoints/{}_{}_criterion.pth'.format(result_dir, name, loss)
        torch.save(weights, filename)

def load_networks(networks, result_dir, name='', loss='', criterion=None):
    weights = networks.state_dict()
    filename = '{}/checkpoints/{}_{}.pth'.format(result_dir, name, loss)
    networks.load_state_dict(torch.load(filename))
    if criterion:
        weights = criterion.state_dict()
        filename = '{}/checkpoints/{}_{}_criterion.pth'.format(result_dir, name, loss)
        criterion.load_state_dict(torch.load(filename))

    return networks, criterion

In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.utils as vutils

import numpy as np
import random

def mix_data(x, use_cuda=True, prob=0.6):
    '''Returns mixed inputs, pairs of targets, and lambda'''

    p = random.uniform(0, 1)

    if p > prob:
        return x

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    fft_1 = torch.fft.fftn(x, dim=(1,2,3))
    abs_1, angle_1 = torch.abs(fft_1), torch.angle(fft_1)

    fft_2 = torch.fft.fftn(x[index, :], dim=(1,2,3))
    abs_2, angle_2 = torch.abs(fft_2), torch.angle(fft_2)

    fft_1 = abs_2*torch.exp((1j) * angle_1)

    mixed_x = torch.fft.ifftn(fft_1, dim=(1,2,3)).float()

    return mixed_x


def train(net, criterion, optimizer, trainloader, epoch=None, **options):
    net.train()
    losses = AverageMeter()

    torch.cuda.empty_cache()
    
    loss_all = 0
    for batch_idx, (data, labels) in enumerate(trainloader):
        if options['use_gpu']:
            inputs, targets = data.cuda(), labels.cuda()
        else:
            inputs, targets = data, labels

        inputs_mix = mix_data(inputs,use_cuda = False)
        inputs_mix = Variable(inputs_mix)
        batch_size = inputs.size(0)
        inputs, inputs_mix = normalize(inputs), normalize(inputs_mix)

        inputs = torch.cat([inputs, inputs_mix], 0)

        with torch.set_grad_enabled(True):
            optimizer.zero_grad()

            _, y = net(inputs, True)
            loss = criterion(y[:batch_size], targets) + criterion(y[batch_size:], targets)
            
            loss.backward()
            optimizer.step()
        
        losses.update(loss.item(), targets.size(0))

        if (batch_idx+1) % options['print_freq'] == 0:
            print("Batch {}/{}\t Loss {:.6f} ({:.6f})" \
                  .format(batch_idx+1, len(trainloader), losses.val, losses.avg))
        
        loss_all += losses.avg

    return loss_all

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value.
       
       Code imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class Logger(object):
    """
    Write console output to external text file.
    
    Code imported from https://github.com/Cysu/open-reid/blob/master/reid/utils/logging.py.
    """
    def __init__(self, fpath=None):
        self.console = sys.stdout
        self.file = None
        if fpath is not None:
            mkdir_if_missing(os.path.dirname(fpath))
            self.file = open(fpath, 'w')

    def __del__(self):
        self.close()

    def __enter__(self):
        pass

    def __exit__(self, *args):
        self.close()

    def write(self, msg):
        self.console.write(msg)
        if self.file is not None:
            self.file.write(msg)

    def flush(self):
        self.console.flush()
        if self.file is not None:
            self.file.flush()
            os.fsync(self.file.fileno())

    def close(self):
        self.console.close()
        if self.file is not None:
            self.file.close()

In [ ]:
options = {
    'data': './',
    'outf': './results',
    'dataset': 'balls',
    'workers': 8,
    'batch_size': 16,
    'lr': 0.1,
    'max_epoch': 10,
    'stepsize': 30,
    'aug': 'none',
    'model': 'densenet',
    'eval_freq': 2,
    'print_freq': 100,
    'gpu': '0',
    'seed': 0,
    'use_cpu': False,
    'eval': False,
    'epsilon': 0.0157,
    'alpha': 0.00784,
    'k': 10,
    'perturbation_type': 'linf'
}


In [ ]:
if 'balls' == options['dataset']:
        Data = Balls(dataroot=options['data'], batch_size=options['batch_size'], _transforms=options['aug'], _eval=options['eval'])
        OODData = Balls2(dataroot=options['data'], batch_size=options['batch_size'], _transforms=options['aug'])

In [ ]:
trainloader, testloader, outloader = Data.train_loader, Data.test_loader, OODData.test_loader
num_classes = Data.num_classes


'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=30):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, rf=False, _eval=False):
        if _eval:
            self.eval()
        else:
            self.train()

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        y = self.linear(out)
        if rf:
            return out, y
        return y


def ResNet18(num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])

In [ ]:
print("Creating model: {}".format(options['model']))
if 'wide_resnet' in options['model']:
        print('wide_resnet')
        net = WideResNet(40, num_classes, 2, 0.0)
elif 'allconv' in options['model']:
        print('allconv')
        net = AllConvNet(num_classes)
elif 'densenet' in options['model']:
        print('densenet')
        net = densenet(num_classes=num_classes)
elif 'resnext' in options['model']:
        print('resnext29')
        net = resnext29(num_classes)
else:
        print('resnet18')
        net = ResNet18(num_classes=num_classes)


Creating model: densenet
densenet


In [ ]:
torch.manual_seed(options['seed'])
os.environ['CUDA_VISIBLE_DEVICES'] = options['gpu']
use_gpu = torch.cuda.is_available()
if options['use_cpu']: use_gpu = False

options.update({'use_gpu': use_gpu})
options['use_gpu']

True

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()

if use_gpu:
        net = nn.DataParallel(net, device_ids=[i for i in range(len(options['gpu'].split(',')))]).cuda()
        criterion = criterion.cuda()

In [ ]:
file_name = '{}_{}_{}'.format(options['model'], options['dataset'], options['aug'])


In [ ]:
params_list = [{'params': net.parameters()},
                {'params': criterion.parameters()}]

In [ ]:
optimizer = torch.optim.SGD(params_list, lr=options['lr'], momentum=0.9, nesterov=True, weight_decay=5e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, gamma=0.2, milestones=[60, 120, 160, 190])

start_time = time.time()

best_acc = 0.0

In [ ]:
for epoch in range(options['max_epoch']):
        print("==> Epoch {}/{}".format(epoch+1, options['max_epoch']))

        train(net, criterion, optimizer, trainloader, epoch=epoch, **options)

        if options['eval_freq'] > 0 and (epoch+1) % options['eval_freq'] == 0 or (epoch+1) == options['max_epoch'] or epoch > 160:
            print("==> Test")
            results = test(net, criterion, testloader, outloader, epoch=epoch, **options)

            if best_acc < results['ACC']:
                best_acc = results['ACC']
                print("Best Acc (%): {:.3f}\t".format(best_acc))
            
            save_networks(net, options['outf'], file_name, criterion=criterion)

        scheduler.step()

elapsed = round(time.time() - start_time)
elapsed = str(datetime.timedelta(seconds=elapsed))
print("Finished. Total elapsed time (h:m:s): {}".format(elapsed))

In [ ]:
net, criterion = load_networks(net, options['outf'], file_name, criterion=criterion)
outloaders = Data.out_loaders
results = test(net, criterion, testloader, outloader, epoch=0, **options)
acc = results['ACC']
res = dict()
res['ACC'] = dict()
acc_res = []
for key in Data.out_keys:
  results = test_robustness(net, criterion, outloaders[key], epoch=0, label=key, **options)
  print('{} (%): {:.3f}\t'.format(key, results['ACC']))
  res['ACC'][key] = results['ACC']
  acc_res.append(results['ACC'])
print('Mean ACC:', np.mean(acc_res))
print('Mean Error:', 100-np.mean(acc_res))

In [ ]:
options['aug'] ='aprs'

print("Creating model: {}".format(options['model']))
if 'wide_resnet' in options['model']:
        print('wide_resnet') 
        net = WideResNet(40, num_classes, 2, 0.0)
elif 'allconv' in options['model']:
        print('allconv')
        net = AllConvNet(num_classes)
elif 'densenet' in options['model']:
        print('densenet')
        net = densenet(num_classes=num_classes)
elif 'resnext' in options['model']:
        print('resnext29')
        net = resnext29(num_classes)
else:
        print('resnet18')
        net = ResNet18(num_classes=num_classes)

torch.manual_seed(options['seed'])
os.environ['CUDA_VISIBLE_DEVICES'] = options['gpu']
use_gpu = torch.cuda.is_available()
if options['use_cpu']: use_gpu = False

options.update({'use_gpu': use_gpu})
options['use_gpu']

criterion = nn.CrossEntropyLoss().cuda()

if use_gpu:
        net = nn.DataParallel(net, device_ids=[i for i in range(len(options['gpu'].split(',')))]).cuda()
        criterion = criterion.cuda()
file_name = '{}_{}_{}'.format(options['model'], options['dataset'], options['aug'])
params_list = [{'params': net.parameters()},
                {'params': criterion.parameters()}]
optimizer = torch.optim.SGD(params_list, lr=options['lr'], momentum=0.9, nesterov=True, weight_decay=5e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, gamma=0.2, milestones=[60, 120, 160, 190])

start_time = time.time()

best_acc = 0.0

In [ ]:
for epoch in range(options['max_epoch']):
        print("==> Epoch {}/{}".format(epoch+1, options['max_epoch']))

        train(net, criterion, optimizer, trainloader, epoch=epoch, **options)

        if options['eval_freq'] > 0 and (epoch+1) % options['eval_freq'] == 0 or (epoch+1) == options['max_epoch'] or epoch > 160:
            print("==> Test")
            results = test(net, criterion, testloader, outloader, epoch=epoch, **options)

            if best_acc < results['ACC']:
                best_acc = results['ACC']
                print("Best Acc (%): {:.3f}\t".format(best_acc))
            
            save_networks(net, options['outf'], file_name, criterion=criterion)

        scheduler.step()

elapsed = round(time.time() - start_time)
elapsed = str(datetime.timedelta(seconds=elapsed))
print("Finished. Total elapsed time (h:m:s): {}".format(elapsed))

In [ ]:
net, criterion = load_networks(net, options['outf'], file_name, criterion=criterion)
outloaders = Data.out_loaders
results = test(net, criterion, testloader, outloader, epoch=0, **options)
acc = results['ACC']
res = dict()
res['ACC'] = dict()
acc_res = []
for key in Data.out_keys:
  results = test_robustness(net, criterion, outloaders[key], epoch=0, label=key, **options)
  print('{} (%): {:.3f}\t'.format(key, results['ACC']))
  res['ACC'][key] = results['ACC']
  acc_res.append(results['ACC'])
print('Mean ACC:', np.mean(acc_res))
print('Mean Error:', 100-np.mean(acc_res))